# Churn Metrics

Adding metrics to people dataset for modeling

In [103]:
import numpy as np
import pandas as pd
import re
import matplotlib
import seaborn as sns

Read the events and people data into dataframes

In [104]:
events = pd.read_csv('Cleaned_Dataset_With_ID.csv', low_memory=False)
people = pd.read_csv('cleaned_people.csv', low_memory= False)

# Adding Number of Ratings Metric

In [105]:
def ratings(events_df, people_df):
    #creating dictionary with distinct id and number of ratings
    number_ratings_dict = events_df[events_df['name_x'] == 'Rating Flow - Complete'].groupby('distinct_id').count()['name_x'].to_dict()
    
    #creating df with just distinct_id and mapping ratings to each distinct id, filling in Nans with 0
    short_df = people_df[['distinct_id']]
    short_df['number_ratings'] = short_df['distinct_id'].map(number_ratings_dict)
    short_df = short_df.fillna(0)
    
    # drop 'distinct_id' column from short_df and then append to the original dataframe
    short_df = short_df.drop(columns = ['distinct_id'])
    people_df = pd.concat([people_df, short_df], axis=1)
    people_df = people_df.drop(columns = ['Unnamed: 0'])
    
    return people_df

In [106]:
ratings_df = ratings(events, people)

/Users/henrycheong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


# SQS

In [107]:
# weight dictionary

weight_dict = {
    'Basic' : 1,
    'Profile' : 2,
    'Explore' : 5,
    'Menu' : 5,
    'Collection' : 7,
    'Rating' : 7,
    'Share' : 10,
    'MISC' : 1,
    'AE' : 1
}

In [108]:
# events dictionary

event_dict = {
    'Basic' : ['Login', 'Logout', 'AppOpen', 'Onboarding - Allow Location', 'Onboarding - Allow Notifications', 
               'Onboarding - Always Allow Location', 'PageVisit', 'Permissions for Locations', 
               'Permissions for Notifications', 'Signup - Sign In', 'Signup - Skip', 'Signup Success', 
               'Visit Signup/Login Page',  'Visit Home Page'],
    
    'Profile' : ['Password Changed', 'Profile - Change Section', 'Profile - Go to Collection', 
                 'Profile - Go to Menu', 'Profile Updated', 'Sidebar - Change Password', 
                 'Sidebar - Contacts Permissions', 'Sidebar - Edit Profile', 'Sidebar - Go to Followers', 
                 'Sidebar - Privacy Policy', 'Sidebar - Sign Out', 'Sidebar - Terms and Conditions', 
                 'Visit Change Password Page', 'Visit Edit Profile Page', 'Followers Page - Go to Profile', 
                 'Item Page - Go to Profile'],
    
    'Explore' : ['Explore - Tap Collection', 'Explore - Visit Explore Page', 'Home - Explore Nearby', 
                 'Feed - Bookmark Icon', 'Feed - Maps Icon', 'Feed - Scroll Down', 'Feed - Tap Dish', 'Item Page - Bookmark', 
                 'Landing - Go to Delivery Feed', 'Landing - Go to Walking Feed', 'Search - Change Section', 
                 'Search - Follow User', 'Search - Invalid Location', 'Search - Results Loaded', 'Search - Tap Home Bar', 
                 'Search - Tap Nearby Restaurant', 'Search - Tap Result', 'Search - Tap Return', 'Search - Tap to Request', 
                 'Search - Unfollow User', 'Tap Searched Dish', 'Visit Item Page', 
                 'Visit Menu Page'],
    
    'Menu' : ['Maps Popup - Go to Reviews', 'Maps Popup - Maybe Later', 'Maps Popup - Open', 'Menu Page - Accuracy Popup', 
              'Menu Page - Add New Dish', 'Menu Page - Collection via Add Icon', 'Menu Page - Filter Items', 
              'Menu Page - Go to Item Page', 'Menu Page - Rate via Add Icon', 'Menu Page - See Restaurant Details', 
              'Menu Page - Submit Flag', 'Menu Page - Switch Menus', 'Menu Page - Tap Flag Icon', 'Visit Menu Page', 
              'Nearby - Bookmark Dish', 'Nearby - Delivery Link', 'Delivery Page - Visit For 10 Sec', 'Home - Order Delivery', 
              'Nearby - Open Maps', 'Nearby - Tap Dish Maps', 'Visit Carousel Wise Dish List'],
    
    'Rating' : ['Edit Rating - Add Photo', 'Rating Flow - Complete', 'Rating Flow - Rate', 
                'Rating Flow - Resubmit', 'Rating Flow - Start', 'Request Restaurant', 
                'Edit Rating - Delete', 'Edit Rating - Edit', 'Edit Rating - Exit', 
                'Edit Rating - Modify Date', 'Item Page - Open My Rating', 
                'Item Page - Tap Upload Photo', 'Profile - Edit Rating Popup'],
    
    'Collection' : ['Collection - Tap Follow Button', 'Collection - Tap Restaurant', 'Collection - Tap Return Button', 
                    'Collection - Tap Search Bar', 'Collection - Visit Collection Detail Page', 
                    'Collection Flow - Create New Collection', 'Collection Flow - Tap Collection', 
                    'Collection Flow - Tap New Collection'], 
    'Share' : ['Share Page - Instagram Story', 'Share Page - Skip', 'Provided App Feedback', 
               'Sidebar - Go to Feedback', 'Sidebar - Invite a Friend',  'Visit App Feedback Page', 
               'Invite Page - Successful Text', 'Search - Invite Button'],
    
    'AE' : ['$ae_crashed', '$ae_first_open', '$ae_session', '$ae_updated', '$campaign_bounced', 
            '$campaign_delivery', '$message_suppressed', '$unsubscribe'],
    
    'MISC': ['DMV-resident'],
}

In [109]:
# short_df only includes the event name and an empty column for category

short_df = events[['name_x']]
short_df.loc['category'] = ''

/Users/henrycheong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [110]:
# create a reverse events dictionary in order to use map function 

reverse_dict = {}
for key, value in event_dict.items():
    for event in value:
        reverse_dict.setdefault(event, []).append(key)
        
# apply reverse dictionary to new column called category that corresponds to the event name
short_df['category'] = short_df['name_x'].map(reverse_dict)

/Users/henrycheong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [111]:
# get the first item in category because each value is a list of one element

short_df['category'] = short_df['category'].str[0]

/Users/henrycheong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [112]:
# map the weights for each category to a score column

short_df['score'] = short_df['category'].map(weight_dict)

/Users/henrycheong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [113]:
# MERGE DATAFRAMES

# drop 'name_x' column from short_df and then append to the original dataframe

labeled_df = short_df.drop(columns = ['name_x'])
event_df = pd.concat([events, labeled_df], axis = 1)
event_df = event_df.drop(columns = ['Unnamed: 0'])

# filter out sessions with -1 for session_id

event_df = event_df[event_df['session_id'] != -1]

# groupby distinct id and then session id 

group_user_session = event_df.groupby(['distinct_id', 'session_id'], as_index=False).agg({'name_x' : 'count', 'score' : 'sum'})
group_user_session = group_user_session.rename(columns = {'name_x' : 'event_count', 'score' : 'SQS'})

# add up all scores for one person and divide by number of sessions

SQS = group_user_session.groupby(['distinct_id'], as_index=False).agg({'SQS' : np.mean})

# merge SQS with original dataframe

SQS_df = pd.merge(left=event_df, right=SQS, on='distinct_id')
#SQS_df = SQS_df.drop(columns = ['Unnamed: 0'])

## Final Dataframe with SQS

In [114]:
SQS_df

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,US,Facebook,Foodie,Google,Unknown,session_id,time_delta,category,score,SQS
0,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Basic,1.0,40.181818
1,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Basic,1.0,40.181818
2,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,AE,1.0,40.181818
3,Signup Success,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:06,-1.0,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,2.0,64.0,Basic,1.0,40.181818
4,Onboarding - Allow Location,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:17,-1.0,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,2.0,64.0,Basic,1.0,40.181818
5,Onboarding - Allow Notifications,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:19,-1.0,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,2.0,64.0,Basic,1.0,40.181818
6,Feed - Scroll Down,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:20,-1.0,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,2.0,64.0,Explore,5.0,40.181818
7,Search - Tap Home Bar,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:21,-1.0,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,2.0,64.0,Explore,5.0,40.181818
8,Search - Tap Return,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:25,-1.0,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,2.0,64.0,Explore,5.0,40.181818
9,Search - Results Loaded,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:26,-1.0,Palo Alto,California,US,-1.0,-1.0,NaN,...,1.0,0.0,0.0,0.0,0.0,2.0,64.0,Explore,5.0,40.181818


# Merge SQS with Ratings

In [115]:
def merge_SQS_rating(SQS, ratings):
    SQS_short = SQS[['distinct_id', 'SQS']]
    df = pd.merge(ratings, SQS_short, left_on = 'distinct_id', right_on = 'distinct_id', how = 'left')
    df = df.drop_duplicates('distinct_id')
    return df

In [116]:
# merged SQS with ratings

result = merge_SQS_rating(SQS_df, ratings_df)
result

,distinct_id,ae_total_app_session_length,name,country_code,auth_provider,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,number_ratings,SQS
0,hr@gmail.com_87,345721,Harshil Raval,IN,Foodie,1759,Asia/Kolkata,2020-03-03 06:08:32+00:00,2.5.0,13.2.3,"iPhone9,3",3.4.9,2019-05-30 01:08:51.984962560+00:00,278 days 04:59:40.015037440,196.544059,130.0,29.951417
1702,E2D9AB63-1718-4D1E-936F-10BE1D889989,4412,Theodore Wu,US,Google,59,America/Los_Angeles,2019-09-08 03:15:47+00:00,2.0.5,13.1,"iPhone9,1",3.4.4,2019-04-22 20:34:46+00:00,138 days 06:41:01.000000000,74.779661,0.0,33.000000
1729,vaibhavverma9@gmail.com_56,71430,Vaibhav Verma,IN,Foodie,621,Asia/Kolkata,2019-04-15 15:18:25+00:00,1.3.7,12.2,"iPhone7,1",3.3.3,2019-05-30 01:08:51.984962560+00:00,-45 days +14:09:33.015037440,115.024155,0.0,NaN
1730,0C954A3F-0AB6-4D12-B4E9-D916C95B0C6E,12836,Tiffany Qi,US,Facebook,77,America/Los_Angeles,2019-11-09 18:33:06+00:00,2.1.6,13.1.3,"iPhone9,3",3.4.4,2019-05-01 06:57:08+00:00,192 days 11:35:58.000000000,166.701299,10.0,55.464286
2062,addidas23@gmail.com_139,2066,Nathan Nangia,US,Foodie,39,America/Chicago,2020-03-18 15:09:21+00:00,2.5.4,13.4,"iPhone11,2",3.4.9,2019-05-30 01:08:51.984962560+00:00,293 days 14:00:29.015037440,52.974359,3.0,18.000000
2148,582FE9F6-5A27-4FC0-8F7D-CCEA4AE76AF8,11594,Alicia BurnsWright,US,Facebook,127,America/New_York,2020-03-11 15:57:07+00:00,2.5.0,13.3.1,"iPhone10,1",3.4.9,2019-10-14 05:49:26+00:00,149 days 10:07:41.000000000,91.291339,56.0,26.086614
2808,7466E480-A994-4CFC-BAFD-C9B96841A8A3,33866,Simmer Team,US,Foodie,372,America/New_York,2020-03-05 00:12:53+00:00,2.5.0,12.4.1,"iPhone10,3",3.4.9,2019-07-11 22:20:45+00:00,237 days 01:52:08.000000000,91.037634,48.0,16.806268
4309,B3A9B684-A566-4DB7-B57F-0C8642031E23,2104,Vinodh Peddi,US,Unknown,38,America/Los_Angeles,2020-03-11 20:43:22+00:00,2.5.0,13.3.1,"iPhone11,6",3.4.9,2019-04-23 21:00:16+00:00,322 days 23:43:06.000000000,55.368421,0.0,9.200000
4359,harshil1088@gmail.com_51,18358,Harshil Raval,IN,Google,123,Asia/Kolkata,2020-02-18 10:45:32+00:00,2.4.7,13.2.3,"iPhone9,3",3.4.4,2019-05-30 01:08:51.984962560+00:00,264 days 09:36:40.015037440,149.252033,12.0,21.611111
4781,6D7E6967-8074-4EF4-B812-E456E91DADB5,3805,Will Bewley,US,Google,62,America/Los_Angeles,2020-03-11 19:23:24+00:00,2.5.0,13.3.1,"iPhone12,5",3.4.9,2018-12-18 13:11:46+00:00,449 days 06:11:38.000000000,61.370968,0.0,27.684211


# Add Slope and Merge Slope with People

In [117]:
df = events.loc[(events['name_x'] == '$ae_session') & (events['time_delta'] != 0)]
df

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
19,19,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:51,65.8,Palo Alto,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,2,64.0
21,21,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:51:52,59.9,Palo Alto,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,3,7.0
24,24,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 14:38:30,72.3,Hayward,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,4,2728.0
36,36,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:05:10,151.9,San Jose,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,5,1466.0
50,50,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:08:11,115.8,San Jose,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,6,75.0
53,53,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:08:59,47.3,San Jose,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,7,6.0
60,60,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:50:32,42.9,Los Altos,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,8,67250.0
69,69,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:51:21,37.9,Los Altos,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,9,13.0
78,78,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:53:39,101.1,Los Altos,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,10,49.0
98,98,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:57:06,186.4,Los Altos,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,11,31.0


In [118]:
def best_fit_slope(xs,ys):
    m = (((np.mean(xs)*np.mean(ys)) - np.mean(xs*ys)) /
         ((np.mean(xs)*np.mean(xs)) - np.mean(xs*xs)))
    return m

In [119]:
def add_slope(df):
    grouped = df.groupby('distinct_id', as_index=False).agg({'time' : 'count'})
    grouped['slope'] = None
    for index, row in grouped.iterrows():
        x = df.loc[df['distinct_id'] == row['distinct_id']]
        length = np.arange(len(x))
        slope = best_fit_slope(length, x['time_delta'])
        grouped.loc[grouped['distinct_id'] == row['distinct_id'], 'slope'] = slope
    final = grouped.drop(columns='time', axis=1)
    return final

In [120]:
# Metric: slope
# AKA: best fit slope of trend of time between session
# Given the trend of time between sessions over time, find the slope of the best fit line for that user. 

slope = add_slope(df)
slope

/Users/henrycheong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


,distinct_id,slope
0,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,1113.36
1,001210DC-54C8-43AD-B295-148F47818391,-11716
2,0025A8A4-2590-4ECA-8CE8-419D710AE46F,615550
3,004C9C76-A530-422A-BA1B-667CDA12A08D,-44.5
4,005144D7-BD3F-405B-9621-5FACA6B1AC04,NaN
5,00708931-5C43-412D-9ECB-6A8839F38179,7
6,00782604-A3A7-4434-8167-3C05AD24637D,1.30126e+06
7,00861508-53FF-4D0A-BB62-D9B35381886C,NaN
8,00CE2C99-8AC3-433E-8611-944CAC725EA5,4.01562e+06
9,00D88292-02E5-4966-8079-DB96622C3970,21540.1


In [121]:
result = result.merge(slope, how='left', on='distinct_id')
result.head(10)

,distinct_id,ae_total_app_session_length,name,country_code,auth_provider,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,number_ratings,SQS,slope
0,hr@gmail.com_87,345721,Harshil Raval,IN,Foodie,1759,Asia/Kolkata,2020-03-03 06:08:32+00:00,2.5.0,13.2.3,"iPhone9,3",3.4.9,2019-05-30 01:08:51.984962560+00:00,278 days 04:59:40.015037440,196.544059,130.0,29.951417,60.7808
1,E2D9AB63-1718-4D1E-936F-10BE1D889989,4412,Theodore Wu,US,Google,59,America/Los_Angeles,2019-09-08 03:15:47+00:00,2.0.5,13.1,"iPhone9,1",3.4.4,2019-04-22 20:34:46+00:00,138 days 06:41:01.000000000,74.779661,0.0,33.000000,1.73002e+06
2,vaibhavverma9@gmail.com_56,71430,Vaibhav Verma,IN,Foodie,621,Asia/Kolkata,2019-04-15 15:18:25+00:00,1.3.7,12.2,"iPhone7,1",3.3.3,2019-05-30 01:08:51.984962560+00:00,-45 days +14:09:33.015037440,115.024155,0.0,NaN,NaN
3,0C954A3F-0AB6-4D12-B4E9-D916C95B0C6E,12836,Tiffany Qi,US,Facebook,77,America/Los_Angeles,2019-11-09 18:33:06+00:00,2.1.6,13.1.3,"iPhone9,3",3.4.4,2019-05-01 06:57:08+00:00,192 days 11:35:58.000000000,166.701299,10.0,55.464286,17473
4,addidas23@gmail.com_139,2066,Nathan Nangia,US,Foodie,39,America/Chicago,2020-03-18 15:09:21+00:00,2.5.4,13.4,"iPhone11,2",3.4.9,2019-05-30 01:08:51.984962560+00:00,293 days 14:00:29.015037440,52.974359,3.0,18.000000,1400.98
5,582FE9F6-5A27-4FC0-8F7D-CCEA4AE76AF8,11594,Alicia BurnsWright,US,Facebook,127,America/New_York,2020-03-11 15:57:07+00:00,2.5.0,13.3.1,"iPhone10,1",3.4.9,2019-10-14 05:49:26+00:00,149 days 10:07:41.000000000,91.291339,56.0,26.086614,2875.54
6,7466E480-A994-4CFC-BAFD-C9B96841A8A3,33866,Simmer Team,US,Foodie,372,America/New_York,2020-03-05 00:12:53+00:00,2.5.0,12.4.1,"iPhone10,3",3.4.9,2019-07-11 22:20:45+00:00,237 days 01:52:08.000000000,91.037634,48.0,16.806268,-648.376
7,B3A9B684-A566-4DB7-B57F-0C8642031E23,2104,Vinodh Peddi,US,Unknown,38,America/Los_Angeles,2020-03-11 20:43:22+00:00,2.5.0,13.3.1,"iPhone11,6",3.4.9,2019-04-23 21:00:16+00:00,322 days 23:43:06.000000000,55.368421,0.0,9.200000,239841
8,harshil1088@gmail.com_51,18358,Harshil Raval,IN,Google,123,Asia/Kolkata,2020-02-18 10:45:32+00:00,2.4.7,13.2.3,"iPhone9,3",3.4.4,2019-05-30 01:08:51.984962560+00:00,264 days 09:36:40.015037440,149.252033,12.0,21.611111,-2055.98
9,6D7E6967-8074-4EF4-B812-E456E91DADB5,3805,Will Bewley,US,Google,62,America/Los_Angeles,2020-03-11 19:23:24+00:00,2.5.0,13.3.1,"iPhone12,5",3.4.9,2018-12-18 13:11:46+00:00,449 days 06:11:38.000000000,61.370968,0.0,27.684211,19991.8


# Add Average Session Time Metric #

In [122]:
# Metric: avg_session_duration
# Similar to "average_session_time", but NOT calculated the same way.
# Takes the average of all of each user's session durations.

def avg_session_time(people, events):
    events = events[events['ae_session_length'] != -1.0]
    groupby_id = events.groupby(['distinct_id']).agg(np.mean)
    people_m2 = people.merge(groupby_id['ae_session_length'], on = 'distinct_id', how = 'left')
    people_m2['ae_session_length'] = people_m2['ae_session_length'].fillna(-1)
    people_m2 = people_m2.rename(columns={"ae_session_length": "avg_session_duration"})
    return people_m2

In [123]:
result = avg_session_time(result, events)
result

,distinct_id,ae_total_app_session_length,name,country_code,auth_provider,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,number_ratings,SQS,slope,avg_session_duration
0,hr@gmail.com_87,345721,Harshil Raval,IN,Foodie,1759,Asia/Kolkata,2020-03-03 06:08:32+00:00,2.5.0,13.2.3,"iPhone9,3",3.4.9,2019-05-30 01:08:51.984962560+00:00,278 days 04:59:40.015037440,196.544059,130.0,29.951417,60.7808,263.902429
1,E2D9AB63-1718-4D1E-936F-10BE1D889989,4412,Theodore Wu,US,Google,59,America/Los_Angeles,2019-09-08 03:15:47+00:00,2.0.5,13.1,"iPhone9,1",3.4.4,2019-04-22 20:34:46+00:00,138 days 06:41:01.000000000,74.779661,0.0,33.000000,1.73002e+06,57.733333
2,vaibhavverma9@gmail.com_56,71430,Vaibhav Verma,IN,Foodie,621,Asia/Kolkata,2019-04-15 15:18:25+00:00,1.3.7,12.2,"iPhone7,1",3.3.3,2019-05-30 01:08:51.984962560+00:00,-45 days +14:09:33.015037440,115.024155,0.0,NaN,NaN,-1.000000
3,0C954A3F-0AB6-4D12-B4E9-D916C95B0C6E,12836,Tiffany Qi,US,Facebook,77,America/Los_Angeles,2019-11-09 18:33:06+00:00,2.1.6,13.1.3,"iPhone9,3",3.4.4,2019-05-01 06:57:08+00:00,192 days 11:35:58.000000000,166.701299,10.0,55.464286,17473,84.050000
4,addidas23@gmail.com_139,2066,Nathan Nangia,US,Foodie,39,America/Chicago,2020-03-18 15:09:21+00:00,2.5.4,13.4,"iPhone11,2",3.4.9,2019-05-30 01:08:51.984962560+00:00,293 days 14:00:29.015037440,52.974359,3.0,18.000000,1400.98,66.258824
5,582FE9F6-5A27-4FC0-8F7D-CCEA4AE76AF8,11594,Alicia BurnsWright,US,Facebook,127,America/New_York,2020-03-11 15:57:07+00:00,2.5.0,13.3.1,"iPhone10,1",3.4.9,2019-10-14 05:49:26+00:00,149 days 10:07:41.000000000,91.291339,56.0,26.086614,2875.54,91.729134
6,7466E480-A994-4CFC-BAFD-C9B96841A8A3,33866,Simmer Team,US,Foodie,372,America/New_York,2020-03-05 00:12:53+00:00,2.5.0,12.4.1,"iPhone10,3",3.4.9,2019-07-11 22:20:45+00:00,237 days 01:52:08.000000000,91.037634,48.0,16.806268,-648.376,91.651567
7,B3A9B684-A566-4DB7-B57F-0C8642031E23,2104,Vinodh Peddi,US,Unknown,38,America/Los_Angeles,2020-03-11 20:43:22+00:00,2.5.0,13.3.1,"iPhone11,6",3.4.9,2019-04-23 21:00:16+00:00,322 days 23:43:06.000000000,55.368421,0.0,9.200000,239841,32.893333
8,harshil1088@gmail.com_51,18358,Harshil Raval,IN,Google,123,Asia/Kolkata,2020-02-18 10:45:32+00:00,2.4.7,13.2.3,"iPhone9,3",3.4.4,2019-05-30 01:08:51.984962560+00:00,264 days 09:36:40.015037440,149.252033,12.0,21.611111,-2055.98,104.029167
9,6D7E6967-8074-4EF4-B812-E456E91DADB5,3805,Will Bewley,US,Google,62,America/Los_Angeles,2020-03-11 19:23:24+00:00,2.5.0,13.3.1,"iPhone12,5",3.4.9,2018-12-18 13:11:46+00:00,449 days 06:11:38.000000000,61.370968,0.0,27.684211,19991.8,49.894737


# Cleaning #

In [124]:
# number_ratings: NaNs imputed with 0.0
# SQS: NaNs imputed with -1.0
# slope: NaNs imputed with 0.0
# avg_session_duration: NaNs i,puted with -1.0

result = result.drop(columns=['average_session_time'])
result['SQS'] = result['SQS'].fillna(-1)
result['slope'] = result['slope'].fillna(0)
result

,distinct_id,ae_total_app_session_length,name,country_code,auth_provider,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,number_ratings,SQS,slope,avg_session_duration
0,hr@gmail.com_87,345721,Harshil Raval,IN,Foodie,1759,Asia/Kolkata,2020-03-03 06:08:32+00:00,2.5.0,13.2.3,"iPhone9,3",3.4.9,2019-05-30 01:08:51.984962560+00:00,278 days 04:59:40.015037440,130.0,29.951417,6.078077e+01,263.902429
1,E2D9AB63-1718-4D1E-936F-10BE1D889989,4412,Theodore Wu,US,Google,59,America/Los_Angeles,2019-09-08 03:15:47+00:00,2.0.5,13.1,"iPhone9,1",3.4.4,2019-04-22 20:34:46+00:00,138 days 06:41:01.000000000,0.0,33.000000,1.730018e+06,57.733333
2,vaibhavverma9@gmail.com_56,71430,Vaibhav Verma,IN,Foodie,621,Asia/Kolkata,2019-04-15 15:18:25+00:00,1.3.7,12.2,"iPhone7,1",3.3.3,2019-05-30 01:08:51.984962560+00:00,-45 days +14:09:33.015037440,0.0,-1.000000,0.000000e+00,-1.000000
3,0C954A3F-0AB6-4D12-B4E9-D916C95B0C6E,12836,Tiffany Qi,US,Facebook,77,America/Los_Angeles,2019-11-09 18:33:06+00:00,2.1.6,13.1.3,"iPhone9,3",3.4.4,2019-05-01 06:57:08+00:00,192 days 11:35:58.000000000,10.0,55.464286,1.747298e+04,84.050000
4,addidas23@gmail.com_139,2066,Nathan Nangia,US,Foodie,39,America/Chicago,2020-03-18 15:09:21+00:00,2.5.4,13.4,"iPhone11,2",3.4.9,2019-05-30 01:08:51.984962560+00:00,293 days 14:00:29.015037440,3.0,18.000000,1.400976e+03,66.258824
5,582FE9F6-5A27-4FC0-8F7D-CCEA4AE76AF8,11594,Alicia BurnsWright,US,Facebook,127,America/New_York,2020-03-11 15:57:07+00:00,2.5.0,13.3.1,"iPhone10,1",3.4.9,2019-10-14 05:49:26+00:00,149 days 10:07:41.000000000,56.0,26.086614,2.875537e+03,91.729134
6,7466E480-A994-4CFC-BAFD-C9B96841A8A3,33866,Simmer Team,US,Foodie,372,America/New_York,2020-03-05 00:12:53+00:00,2.5.0,12.4.1,"iPhone10,3",3.4.9,2019-07-11 22:20:45+00:00,237 days 01:52:08.000000000,48.0,16.806268,-6.483760e+02,91.651567
7,B3A9B684-A566-4DB7-B57F-0C8642031E23,2104,Vinodh Peddi,US,Unknown,38,America/Los_Angeles,2020-03-11 20:43:22+00:00,2.5.0,13.3.1,"iPhone11,6",3.4.9,2019-04-23 21:00:16+00:00,322 days 23:43:06.000000000,0.0,9.200000,2.398413e+05,32.893333
8,harshil1088@gmail.com_51,18358,Harshil Raval,IN,Google,123,Asia/Kolkata,2020-02-18 10:45:32+00:00,2.4.7,13.2.3,"iPhone9,3",3.4.4,2019-05-30 01:08:51.984962560+00:00,264 days 09:36:40.015037440,12.0,21.611111,-2.055978e+03,104.029167
9,6D7E6967-8074-4EF4-B812-E456E91DADB5,3805,Will Bewley,US,Google,62,America/Los_Angeles,2020-03-11 19:23:24+00:00,2.5.0,13.3.1,"iPhone12,5",3.4.9,2018-12-18 13:11:46+00:00,449 days 06:11:38.000000000,0.0,27.684211,1.999175e+04,49.894737
